In [ ]:
import ROOT
from pathlib import Path
import lzt_utils as lu
import mplhep
import matplotlib.pyplot as plt
import awkward  as ak
import numpy as np
import matplotlib.pyplot as plt 
ROOT.EnableImplicitMT()
plt.style.use(mplhep.style.ROOT)

Module libc not found.


Welcome to JupyROOT 6.28/02


In [2]:
lzt_data = Path('/', 'root', 'ext_data', 'lorenzetti')
dataset_name = '2024_08_20_21_08_0000000000_zee'
dataset_path = lzt_data / dataset_name
output_dir = lzt_data / 'checks'
output_dir.mkdir(exist_ok=True, parents=True)
dataset = lu.dataset.LztDataset.from_dir(dataset_path)
dataset

LztDataset(path=/root/ext_data/lorenzetti/2024_08_20_21_08_0000000000_zee, label=Z->ee)

# ESD

In [3]:
esd_rdf = dataset.get_esd_rdf(n_files=1)
esd_rdf.Describe()

Dataframe from TChain CollectionTree in file /root/ext_data/lorenzetti/2024_08_20_21_08_0000000000_zee/ESD/zee.ESD.99.root

Property                Value
--------                -----
Columns in total           51
Columns from defines        0
Event loops run             0
Processing slots           10

Column                                          Type                                            Origin
------                                          ----                                            ------
CaloCellContainer_Cells                         ROOT::VecOps::RVec<xAOD::CaloCell_t>            Dataset
CaloCellContainer_Cells.descriptor_link         ROOT::VecOps::RVec<Int_t>                       Dataset
CaloCellContainer_Cells.deta                    ROOT::VecOps::RVec<Float_t>                     Dataset
CaloCellContainer_Cells.dphi                    ROOT::VecOps::RVec<Float_t>                     Dataset
CaloCellContainer_Cells.e                       ROOT::VecOps::RVec<Float_

In [ ]:
ak_cols = [
    "EventInfoContainer_Events.eventNumber",
    "EventInfoContainer_Events.avgmu",
    "CaloCellContainer_Cells.eta",
    "CaloCellContainer_Cells.phi",
    "CaloCellContainer_Cells.e",
    "CaloCellContainer_Cells.et",
]
esd_ak = dataset.get_esd_ak(n_files=1, columns=ak_cols)
esd_ak = esd_ak[ak.argsort(esd_ak['EventInfoContainer_Events.eventNumber'])]
esd_ak

# AOD

In [3]:
aod_rdf = dataset.get_aod_rdf()
aod_rdf.Describe()

Dataframe from TChain CollectionTree in file /root/ext_data/lorenzetti/2024_08_20_21_08_0000000000_zee/AOD/zee.AOD.root

Property                Value
--------                -----
Columns in total           99
Columns from defines        0
Event loops run             0
Processing slots           10

Column                                          Type                                            Origin
------                                          ----                                            ------
CaloCellContainer_Cells                         ROOT::VecOps::RVec<xAOD::CaloCell_t>            Dataset
CaloCellContainer_Cells.descriptor_link         ROOT::VecOps::RVec<Int_t>                       Dataset
CaloCellContainer_Cells.deta                    ROOT::VecOps::RVec<Float_t>                     Dataset
CaloCellContainer_Cells.dphi                    ROOT::VecOps::RVec<Float_t>                     Dataset
CaloCellContainer_Cells.e                       ROOT::VecOps::RVec<Float_t> 

In [4]:
max_evt_num = aod_rdf.Max("EventInfoContainer_Events.eventNumber").GetValue()
min_evt_num = aod_rdf.Min("EventInfoContainer_Events.eventNumber").GetValue()
print(f'Event Number range is {min_evt_num} - {max_evt_num}')

Event Number range is 3.0 - 103167.0


In [5]:
evt_num_to_plot = 1000
evt_num_rdf = aod_rdf \
    .Filter(f'EventInfoContainer_Events.eventNumber[0] <= {evt_num_to_plot + 100}') \
    .Filter(f'EventInfoContainer_Events.eventNumber[0] >= {evt_num_to_plot - 100}')

In [16]:
ak_cols = [
    "EventInfoContainer_Events.eventNumber",
    "EventInfoContainer_Events.avgmu",
    "CaloCellContainer_Cells.eta",
    "CaloCellContainer_Cells.phi",
    "CaloCellContainer_Cells.e",
    "CaloCellContainer_Cells.et",
    'CaloClusterContainer_Clusters.e',
    'CaloClusterContainer_Clusters.et',
    'CaloClusterContainer_Clusters.eta',
    'CaloClusterContainer_Clusters.phi',
    'CaloClusterContainer_Clusters.deta',
    'CaloClusterContainer_Clusters.dphi',
]
evt_num_ak = lu.dataset.aod_rdf_to_ak(
    evt_num_rdf, columns=ak_cols)
evt_num_ak = evt_num_ak[ak.argsort(evt_num_ak['EventInfoContainer_Events.eventNumber'])]
evt_num_ak

<Array [{...}, {...}, {...}, ..., {...}, {...}] type='79 * {"CaloCellContai...'>

In [21]:
for i in evt_num_ak:
    print(i)
    print(type(i))
    break

{'CaloCellContainer_Cells.eta': [-2.37, ..., -2.53], ...}
<class 'awkward.highlevel.Record'>


In [24]:
evt_num_pdf = ak.to_dataframe(evt_num_ak)
evt_num_pdf

CaloCellContainer_Cells.eta  CaloCellContainer_Cells.phi  \
entry subentry                                                             
0     0                             -2.3669                       1.2272   
1     0                             -1.5625                       0.1473   
      1                             -1.5625                       0.2454   
2     0                              2.6199                      -0.6381   
      1                              2.6199                      -0.5399   
...                                     ...                          ...   
76    1                             -1.8159                       1.1290   
77    0                             -2.3669                       2.2089   
      1                             -2.3669                       2.3071   
78    0                             -1.4625                       1.5217   
      1                             -1.4625                       1.6199   

                CaloCellContainer_Cells.e  CaloCellContainer_Cells.et  \
entry subentry                                                          
0     0                         -3.303859                   -0.614211   
1     0                        -44.369423                  -17.817812   
      1                         15.906891                    6.387867   
2     0                         34.061241                    4.933852   
      1                         80.284981                   11.629470   
...                                   ...                         ...   
76    1                          6.216360                    1.970540   
77    0                        -22.714432                   -4.222778   
      1                         20.810328                    3.868791   
78    0                        -28.344395                  -12.463473   
      1                        139.362045                   61.279671   

                CaloClusterContainer_Clusters.e  \
entry subentry                                    
0     0                           140952.953125   
1     0                            37746.242188   
      1                            32129.230469   
2     0                           177850.000000   
      1                            27791.507812   
...                                         ...   
76    1                            29658.380859   
77    0                           225318.625000   
      1                            51494.468750   
78    0                            68027.781250   
      1                            41571.687500   

                CaloClusterContainer_Clusters.et  \
entry subentry                                     
0     0                             22139.156250   
1     0                             15325.409180   
      1                             25012.921875   
2     0                             22917.722656   
      1                             25328.572266   
...                                          ...   
76    1                             29600.548828   
77    0                             35390.281250   
      1                             31794.689453   
78    0                             35642.839844   
      1                             41490.625000   

                CaloClusterContainer_Clusters.eta  \
entry subentry                                      
0     0                                   -2.5380   
1     0                                   -1.5505   
      1                                    0.7375   
2     0                                    2.7380   
      1                                    0.4375   
...                                           ...   
76    1                                    0.0625   
77    0                                   -2.5380   
      1                                   -1.0625   
78    0                                   -1.2625   
      1                                   -0.0625   

                CaloClusterContainer_Clusters.

In [32]:
for i in np.unique(evt_num_ak[1]['CaloCellContainer_Cells.eta']):
    print(i)

-1.7313001
-1.7281001
-1.7255001
-1.7248001
-1.7216
-1.7183001
-1.7159
-1.7151
-1.7146
-1.7118001
-1.7086
-1.7053001
-1.7021
-1.7005001
-1.6988001
-1.6956
-1.6923001
-1.6909001
-1.6891
-1.6865001
-1.6858001
-1.6826
-1.6793001
-1.679
-1.6766
-1.6761
-1.6755
-1.6728001
-1.6696
-1.6663
-1.6659
-1.6631
-1.6598
-1.6566
-1.6533
-1.6505
-1.6501
-1.6468
-1.6436
-1.6408999
-1.6403
-1.6373
-1.6371
-1.6365
-1.6338
-1.6306
-1.6273
-1.6255001
-1.6241
-1.6208
-1.6176
-1.6159
-1.6146
-1.6143
-1.6111
-1.6078
-1.6046
-1.6013
-1.6005001
-1.5981
-1.5948
-1.5916
-1.5909
-1.5883
-1.5865
-1.5850999
-1.5818
-1.5785999
-1.5766001
-1.5755001
-1.5753
-1.5720999
-1.5688
-1.5659001
-1.5655999
-1.5625
-1.5623
-1.5591002
-1.5558
-1.5551
-1.5526001
-1.5519001
-1.5505
-1.5493
-1.5486
-1.5461001
-1.5454001
-1.5428
-1.5422001
-1.5409
-1.5396001
-1.5389001
-1.5374999
-1.5373
-1.5365
-1.5363
-1.5356
-1.5331001
-1.5324001
-1.5297999
-1.5291
-1.5266001
-1.5259001
-1.5255
-1.5232999
-1.5225999
-1.5201001
-1.5194001
-1.51679

In [30]:
for i, (idx, row) in enumerate(evt_num_pdf.iterrows()):
    if i >= 3:
        break
    print(i)
    print(idx)
    print(row)

0
(0, 0)
CaloCellContainer_Cells.eta                  -2.366900
CaloCellContainer_Cells.phi                   1.227200
CaloCellContainer_Cells.e                    -3.303859
CaloCellContainer_Cells.et                   -0.614211
CaloClusterContainer_Clusters.e          140952.953125
CaloClusterContainer_Clusters.et          22139.156250
CaloClusterContainer_Clusters.eta            -2.538000
CaloClusterContainer_Clusters.phi             1.423500
CaloClusterContainer_Clusters.deta            0.200000
CaloClusterContainer_Clusters.dphi            0.200000
EventInfoContainer_Events.eventNumber       901.000000
EventInfoContainer_Events.avgmu               0.000000
Name: (0, 0), dtype: float64
1
(1, 0)
CaloCellContainer_Cells.eta                 -1.562500
CaloCellContainer_Cells.phi                  0.147300
CaloCellContainer_Cells.e                  -44.369423
CaloCellContainer_Cells.et                 -17.817812
CaloClusterContainer_Clusters.e          37746.242188
CaloClusterContainer_Cl